In [8]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  8 00:02:25 2023
@author: Hoai-Nam Nguyen
"""

%run Taiwan_Time.ipynb
%run Authentication.ipynb
%run Retrieve_RSSI.ipynb
import requests
import json
import math
import pandas as pd 
from pymongo import MongoClient
from datetime import timedelta
from datetime import datetime, timedelta
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
warnings.filterwarnings("ignore")
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

# =============================================================================
# Calculate Taiwan time (GMT +8)

ts, ts_tw_str, ts_tw = calculate_taiwan_time()

# =============================================================================
# Decleare variables

username= os.environ.get('ACCOUNT') # Aruba controller username
password= os.environ.get('PASSWORD') # Aruba controller password
vMM_aosip= os.environ.get('vMM_aosip') # Aruba controller IP address
MONGOIP = os.environ.get("MONGOIP")  # MongoDB IP address
ROGUEAPDB = os.environ.get('ROGUEAPDB') # RogueAP database
ROGUEAPCOLLECTION = os.environ.get("ROGUEAPCOLLECTION") # Coordinate collection

df = pd.read_excel('AP_name.xlsx')

# =============================================================================
# Import Aruba AP information from excel

# Create floor list
f = df['Floor'].tolist()

# Create CLI input ap-name list
IY = df.values.tolist()
IY = [[value for value in inner_list[1:] if not (isinstance(value, float) and math.isnan(value))] for inner_list in IY]


# =============================================================================
# Login controller function

token = authentication(username,password,vMM_aosip)

# =============================================================================
# Retrieve and process RSSI for every floor in IY

df0 = rss_retrieve(IY[0], f[0], vMM_aosip, token)
try:
    df1 = rss_retrieve(IY[1], f[1], vMM_aosip, token)
    dfa = df0.append(df1, ignore_index=True)
except Exception:
    pass
try:
    df2 = rss_retrieve(IY[2], f[2], vMM_aosip, token)
    dfa = dfa.append(df2, ignore_index=True)
except Exception:
    pass
try:
    df3 = rss_retrieve(IY[3], f[3], vMM_aosip, token)
    dfa = dfa.append(df3, ignore_index=True)
except Exception:
    pass
try:
    df4 = rss_retrieve(IY[4], f[4], vMM_aosip, token)
    dfa = dfa.append(df4, ignore_index=True)
except Exception:
    pass
try:
    df5 = rss_retrieve(IY[5], f[5], vMM_aosip, token)
    dfa = dfa.append(df5, ignore_index=True)
except Exception:
    pass
try:
    df6 = rss_retrieve(IY[6], f[6], vMM_aosip, token)
    dfa = dfa.append(df6, ignore_index=True)
except Exception:
    pass
try:
    df7 = rss_retrieve(IY[7], f[7], vMM_aosip, token)
    dfa = dfa.append(df7, ignore_index=True)
except Exception:
    pass
try:
    df8 = rss_retrieve(IY[8], f[8], vMM_aosip, token)
    dfa = dfa.append(df8, ignore_index=True)
except Exception:
    pass
try:
    df9 = rss_retrieve(IY[9], f[9], vMM_aosip, token)
    dfa = dfa.append(df9, ignore_index=True)
except Exception:
    pass
try:
    df10 = rss_retrieve(IY[10], f[10], vMM_aosip, token)
    dfa = dfa.append(df10, ignore_index=True)
except Exception:
    pass

# =============================================================================
# Save as json format

data_json = json.loads(dfa.to_json(orient='records'))

# Add datetime
for i in range(len(data_json)):
    data_json[i]['ts'] = ts 
    data_json[i]['DatetimeStr'] = ts_tw_str
    data_json[i]['Datetime'] = ts_tw

# =============================================================================
# Store json data to MongoDB

client = MongoClient(MONGOIP,27017)
db = client[ROGUEAPDB]
col=db[ROGUEAPCOLLECTION]
## Set time to auto-delete MongoDB
#previous_day = datetime.now() - timedelta(days=30) 
# col.delete_many({"Datetime": {"$lt": previous_day}})
col.insert_many(data_json)
print('Done!')

dfa

{'Time-Range night-hours, Periodic': [{'Applied': 'No', 'End-time': '23:59', 'EndDay': None, 'Start-time': '18:01', 'StartDay': 'weekday'}, {'Applied': 'No', 'End-time': '07:59', 'EndDay': None, 'Start-time': '00:00', 'StartDay': 'weekday'}], 'Time-Range working-hours, Periodic': [{'Applied': 'No', 'End-time': '18:00', 'EndDay': None, 'Start-time': '08:00', 'StartDay': 'weekday'}], '_data': [], '_meta': ['StartDay', 'Start-time', 'EndDay', 'End-time', 'Applied']}
Done!


,bssid,essid,ap type,channel,AP01,AP03,AP05,AP07,mon AP number,xloc,yloc,floor,AP09
0,a0:2e:f3:01:d2:0a,5052012052,interfering,1+,-38,None,-53,None,2,,,1F,NaN
1,a0:2e:f3:01:9d:69,5052013132,interfering,1+,-58,None,-23,None,2,,,1F,NaN
2,4c:c5:3e:9c:cc:4d,ABC,interfering,11,-55,None,-50,None,2,,,1F,NaN
3,04:42:1a:d8:f1:fc,ASUS_5G,interfering,161E,None,None,-48,None,1,,,1F,NaN
4,88:de:7c:62:9d:71,IT_SET1,interfering,1,-40,None,None,None,1,,,1F,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,46:1c:a8:68:7f:ee,DIRECT-ee-HP M277 LaserJet,interfering,6,-57,None,None,None,1,,,9F,NaN
100,f0:2f:74:0f:87:18,HCHuang,interfering,6,None,None,None,-56,1,,,9F,NaN
101,bc:0f:9a:66:a9:14,Office_906,interfering,5,None,-40,None,-47,2,,,9F,NaN
102,40:ee:15:8f:51:18,Prof_Wang_office,interfering,4+,-38,NaN,NaN,NaN,1,,,11F,NaN
